In [1]:
from tkinter import *
from tkinter import ttk
import tkinter as tk
from PIL import Image, ImageTk
from pandastable import Table


from src.utils_ml import ml_pipeline

def labels():
    global root
    root = Tk()
    root.geometry('1440x900')
    
    # Background
    global bg_img 
    bg_img = ImageTk.PhotoImage(Image.open('./ui/bgw.png'))
    bg_label = Label(image=bg_img)
    bg_label.pack()

    # Interface Titles
    title_1 = Label(root, text='PyForecast', font=("Helvetica Neue Bold", 36), 
                            anchor='w', fg='black').place(x=600, y=20)
    data_title = Label(root, text='Data', font=("Helvetica Neue Bold", 18), 
                            anchor='w', fg="#666666").place(x=50, y=80)
    data_label = Label(root, text='Select the data', 
                        font=("Helvetica Neue Regular", 13), 
                        anchor='w', fg='black').place(x=50, y=110)
    upload_label = Label(root, text='or upload your own', 
                        font=("Helvetica Neue Regular", 13), 
                         anchor='w', fg='black').place(x=50, y=165)

    #Filter labels and buttons
    filter_title = Label(root, text='Filters', font=("Helvetica Neue Bold", 18), 
                            anchor='w', fg="#666666").place(x=50, y=250)
    season_label = Label(root, text='Select the frequency', 
                        font=("Helvetica Neue Regular", 13), 
                         anchor='w', fg='black').place(x=50, y=280)
    horizon_label = Label(root, text='Input Forecast Horizon', 
                    font=("Helvetica Neue Regular", 13), 
                          anchor='w', fg='black').place(x=50, y=335)
    model_title = Label(root, text='Models', font=("Helvetica Neue Bold", 18), 
                        anchor='w', fg="#666666").place(x=50, y=400)
    metric_title = Label(root, text='Metrics', font=("Helvetica Neue Bold", 18), 
                        anchor='w', fg="#666666").place(x=50,y=515)
    model_sample_title = Label(root, text='Models Sample', font=("Helvetica Neue Bold", 18), 
                        anchor='w', fg="#666666").place(x=360,y=80)
    metric_table_title = Label(root, text='Metrics Table', font=("Helvetica Neue Bold", 18), 
                        anchor='w', fg="#666666").place(x=720,y=515)
    residuals_plot_title = Label(root, text='Residuals', font=("Helvetica Neue Bold", 18), 
                        anchor='w', fg="#666666").place(x=360,y=515)
    global series_img
    series_img = ImageTk.PhotoImage(Image.open('./ui/series_blank.jpg'))
    series_blank = Label(image=series_img)
    series_blank.place(x=350, y=110)

    
def model_buttons():
    #Model Parameters for Checkboxes
    model_widgets = {'Naive': {'bool': IntVar(), 'place': (50, 435)},
                     'SeasonalNaive': {'bool': IntVar(), 'place': (50, 455)}, 
                     'Naive2': {'bool': IntVar(), 'place': (50, 475)},
                     'RandomWalkDrift': {'bool': IntVar(), 'place': (170, 435)}, 
                     'Croston': {'bool': IntVar(), 'place': (170, 455)},
                     'MovingAverage': {'bool': IntVar(), 'place': (170, 475)}}
    
    # Declare check buttons for each model
    for model_name in model_widgets:
        model_widgets[model_name]["check_button"] = Checkbutton(root, text=model_name, 
                                                                fg='black', font=("Helvetica Neue Regular", 13),
                                                                variable=model_widgets[model_name]['bool'])
    #Placing check buttons for each model
    for model_name in model_widgets:
        x, y = model_widgets[model_name]["place"]
        check_button = model_widgets[model_name]["check_button"]
        check_button.place(x=x, y=y)
    
    return model_widgets
    

def metric_buttons():
    #Metric Parameters for Checkboxes
    metric_widgets = {'RMSE': {'bool': IntVar(), 'place':(50, 550)},
                      'MAPE': {'bool': IntVar(), 'place': (50, 570)},
                      'SMAPE': {'bool': IntVar(), 'place': (50, 590)},
                      'MASE': {'bool': IntVar(), 'place': (170, 550)},
                      'RMSSE': {'bool': IntVar(), 'place': (170, 570)}}

    # Declare check buttons for each metric
    for metric_name in metric_widgets:
        metric_widgets[metric_name]["check_button"] = Checkbutton(root, text=metric_name,
                                                                  fg='black', font=("Helvetica Neue Regular", 13),
                                                                  variable=metric_widgets[metric_name]['bool'])
    # Place check buttons for each metric
    for metric_name in metric_widgets:
        x, y = metric_widgets[metric_name]["place"]
        check_button = metric_widgets[metric_name]["check_button"]
        check_button.place(x=x, y=y)
    
    return metric_widgets

def buttons():
    #Data selection button
    data = ['Hourly', 'Daily', 'Weekly', 'Monthly','Quarterly']

    global data_widget
    data_widget = ttk.Combobox(root, value=data, width=15)
    data_widget.place(x=50, y=135)
    
    #Upload button 
    global upload_img
    upload_img = ImageTk.PhotoImage(Image.open('./ui/upload.png'))
    upload_button = Button(root, image=upload_img, padx=0, pady=0)
    #upload_button.config(highlightbackground='white')
    upload_button.place(x=50, y=190)
    
    #Frequency button
    global freq_widget
    freqs = ['H', 'D', 'W', 'M','Q']
    freq_widget = ttk.Combobox(root, value=freqs, width=15)
    freq_widget.place(x=50, y=305)
    
    #Horizon entry
    global horizon_widget
    horizon_widget = Entry(root, width=15)
    horizon_widget.place(x=50, y=360)
    
    global model_widgets
    model_widgets = model_buttons()
    
    global metric_widgets
    metric_widgets = metric_buttons()
    
    global analyze_img
    analyze_img = ImageTk.PhotoImage(Image.open('./ui/analyze.png'))
    analyze_button = Button(root, image=analyze_img, border=0, 
                            command=display,  padx=0, pady=0)
    analyze_button.config(highlightbackground='white')
    analyze_button.place(x=50, y=650)

def display():
    dataset_name, models_filter, metrics_filter = get_run_parameters()
    ml_pipeline(dataset_name=dataset_name,
                num_obs=10,
                models_filter=models_filter,
                metrics_filter=metrics_filter)
    
    global plot_file
    plot_file = ImageTk.PhotoImage(Image.open('./results/grid_series.png'))
    plot_label = Label(image=plot_file)
    plot_label.place(x=350, y=110)
    frame = tk.Frame(root, background='white')
    frame.place(x=720,y=550)
    pt = Table(frame, width=600, height=130)
    pt.columncolors = 'red' 
    pt.importCSV('./results/metrics.csv')
    pt.redraw()
    pt.show()

    
##### Selection Functions

def data_selection():
    return data_widget.get()

def model_selection():
    models_filter = []
    for model_name in model_widgets:
        model_bool = model_widgets[model_name]['bool']
        if model_bool.get() == 1:
            models_filter.append(model_name)
    return models_filter
        
def metric_selection():
    metrics_filter = []
    for metric_name in metric_widgets:
        metric_bool = metric_widgets[metric_name]['bool']
        if metric_bool.get() == 1:
            metrics_filter.append(metric_name)
    return metrics_filter

def get_run_parameters():
    dataset_name = data_selection()
    models_filter = model_selection()
    metrics_filter = metric_selection()
    return dataset_name, models_filter, metrics_filter

#ttk.Progressbar
#bg='#010626'

labels()
buttons()
root.mainloop()


 Fitting models
Naive
SeasonalNaive
Naive2
RandomWalkDrift
Croston
MovingAverage

 Evaluating models
RMSE
MAPE
